In [ ]:
#Temperatur dependent defect document
# General
import numpy as np
import matplotlib.pyplot as plt
from eskild_function import *

# For handling structures and visualizing structures
from ase import Atoms
from ase.build import graphene_nanoribbon
from ase.visualize import view
from ase.visualize.plot import plot_atoms
from ase.io import read, write

# For MD
from ase.calculators.tersoff import Tersoff
from ase.constraints import FixAtoms
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.langevin import Langevin
from ase import units
from ase.neighborlist import neighbor_list

import os
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

In [ ]:
# --- Structure Definition ---
kind = "armchair"     # "armchair" or "zigzag"
n = 5                 # width parameter
length = 1            # periodic repetitions along z, i.e., transport direction here
vacuum = 15.0         # vacuum in non-periodic directions (Å)
bond = 1.43877067     # Optimized using C.tersoff potential discussed below

ribbon = graphene_nanoribbon(n=n,
                             m=length,
                             type=kind,
                             C_C=bond,
                             vacuum=vacuum)
ribbon.pbc = True
ribbon = sort_atoms(ribbon) # Function from eskild_function
structure = ribbon

# --- Hamiltonian Setup (Pristine) ---
# Create a transport structure with enough length to define leads
Ntransport = 5 
pristine_structure = structure.repeat((1, 1, Ntransport))
pos = pristine_structure.positions

# Sort atoms (Standard sorting for this project)
x_sort_args = np.lexsort((pos[:, 0], pos[:, 2]))
if len(x_sort_args) > 10:
    x_sort_args[[9, 10]] = x_sort_args[[10, 9]]
pos_sorted = pos[x_sort_args]

# Parameters for splitting
n_lead_atoms = 20 

# Extract Matrices
H_full = hamiltonian(pos_sorted) 

idx_L = np.arange(n_lead_atoms)
idx_R = np.arange(len(pos_sorted)-n_lead_atoms, len(pos_sorted))
idx_D = np.arange(n_lead_atoms, len(pos_sorted)-n_lead_atoms)

H_L = H_full[np.ix_(idx_L, idx_L)]      
H_D = H_full[np.ix_(idx_D, idx_D)]      
H_R = H_full[np.ix_(idx_R, idx_R)]      
t_L = H_full[np.ix_(idx_D[:n_lead_atoms], idx_L)]      
t_R = H_full[np.ix_(idx_D[-n_lead_atoms:], idx_R)]   

# --- MD Setup ---
timestep = 1.0
calc = Tersoff.from_lammps("C.tersoff")

In [ ]:
def resistance(Transmission):
    # Resistance in units of 1/G0
    # G = G0 * Transmission
    # R = 1/G = 1/(G0 * Transmission) = (1/Transmission) * (1/G0)
    # Returning value in units of 1/G0 means returning 1/Transmission
    return 1.0 / np.array(Transmission)

# Parameters needed for the calculation
# Use astype(int) to ensure we have integers for the loop ranges
ntiles_list = np.linspace(5, 20, 16).astype(int) # Varying lengths (number of unit cell repetitions)

# Simulation parameters
md_temp_K = 300.0
md_nsteps = 500  # Number of MD steps per length (adjust as needed for time vs accuracy)
md_dump = 10     # Interval for saving snapshots
eta = 0.0001
E_fermi = 0.0    # Energy to calculate resistance at

# Pre-compute Self-Energies at Fermi Level (E=0)
z_fermi = E_fermi + 1j * eta
sig_s_L_fermi, _, _, _ = self_energy_decimation(z_fermi, H_L, t_L, iterations=100)
sig_s_R_fermi, _, _, _ = self_energy_decimation(z_fermi, H_R, t_R, iterations=100)

def calculate_T_at_fermi(H_device, sig_L, sig_R):
    """
    Calculate Transmission at Fermi level for a given Device Hamiltonian
    """
    size = H_device.shape[0]
    I = np.eye(size)
    z = E_fermi + 1j * eta
    
    # Construct Effective Hamiltonian
    device_zero = np.zeros((size, size), dtype=complex)
    gray_left = device_zero.copy()
    
    n_lead = sig_L.shape[0]
    gray_left[:n_lead, :n_lead] = sig_L
    
    gray_right = device_zero.copy()
    n_lead_R = sig_R.shape[0]
    gray_right[-n_lead_R:, -n_lead_R:] = sig_R
    
    H_eff = H_device + gray_left + gray_right
    
    # Gamma matrices
    gamma_L = 1j * (gray_left - np.conjugate(np.transpose(gray_left)))
    gamma_R = 1j * (gray_right - np.conjugate(np.transpose(gray_right)))
    
    # Green's Function
    G_eff = np.linalg.inv(z*I - H_eff)
    
    T = np.trace(gamma_L @ G_eff @ gamma_R @ np.conjugate(np.transpose(G_eff)))
    return np.real(T)

def run_simulation_for_length(ntile):
    """
    Runs MD and analyzes transport for a specific nanoribbon length.
    Returns the average resistance for this length.
    """
    # 1. Create Structure for this length
    xyz = structure.positions
    lattice = structure.cell[:]
    tiledir = 2 # z-direction
    
    # Create repeated structure
    for n in range(1, ntile):
        xyz = np.concatenate((xyz, structure.positions + lattice[tiledir, :]*n))
    
    tilemat = np.eye(3, dtype=int)
    tilemat[tiledir, tiledir] = ntile
    lattice_long = tilemat @ lattice
    natoms = len(xyz)
    
    current_md_structure = Atoms(natoms*["C"], positions=xyz, cell=lattice_long, pbc=True)
    
    # Apply Constraints (Leads and Edges)
    natoms_elec = len(structure)
    fixed_uc = 2
    leftinds = list(range(0, natoms_elec*fixed_uc))
    rightinds = list(range(natoms - natoms_elec*fixed_uc, natoms))
    
    cutoff = 1.5
    bulk_nneighbors = 3
    i_list, j_list = neighbor_list("ij", current_md_structure, cutoff)
    counts = np.bincount(i_list, minlength=len(current_md_structure))
    edgeinds = list(np.where(counts < bulk_nneighbors)[0])
    
    allinds = np.unique(leftinds + rightinds + edgeinds)
    current_md_structure.set_constraint(FixAtoms(mask=allinds))
    current_md_structure.calc = calc # Use the Tersoff calculator defined earlier

    # 2. Run MD
    # Use unique filename per process to avoid conflicts
    temp_xyz_file = f"md_len_{ntile}.xyz"
    
    if os.path.exists(temp_xyz_file):
        # print(f"Using existing trajectory: {temp_xyz_file}") # Print might not work well in parallel
        pass
    else:
        MaxwellBoltzmannDistribution(current_md_structure, temperature_K=md_temp_K)
        dyn = Langevin(current_md_structure, timestep*units.fs, temperature_K=md_temp_K, friction=0.01/units.fs, logfile=None)
        dyn.attach(lambda: write(temp_xyz_file, current_md_structure, append=True), interval=md_dump)
        
        # Run simulation
        dyn.run(md_nsteps)
    
    # 3. Analyze Trajectory
    # Use index=":" to read all frames, slicing after reading is safer for file I/O in parallel? 
    # Actually reading everything then slicing is fine.
    traj = read(temp_xyz_file, index=":")
    traj_analysis = traj[int(len(traj)*0.25):]
    
    trans_samples = []
    
    # Processing frames sequentially inside the parallel task to avoid nested parallelism overhead
    for frame in traj_analysis:
        pos = frame.positions
        x_sort_args = np.lexsort((pos[:, 0], pos[:, 2]))
        
        if len(x_sort_args) > 10:
             x_sort_args[[9, 10]] = x_sort_args[[10, 9]]
             
        pos_sorted = pos[x_sort_args]
        
        H_full_frame = hamiltonian(pos_sorted)
        n_lead_atoms = 20
        idx_D = np.arange(n_lead_atoms, len(pos_sorted)-n_lead_atoms)
        H_D_frame = H_full_frame[np.ix_(idx_D, idx_D)]
        
        T = calculate_T_at_fermi(H_D_frame, sig_s_L_fermi, sig_s_R_fermi)
        trans_samples.append(T)
    
    # Clean up file
    # if os.path.exists(temp_xyz_file):
    #     os.remove(temp_xyz_file)

    R_quantum = 12906.0
    R_samples = (1.0 / np.array(trans_samples)) * R_quantum
    return np.mean(R_samples)

print(f"Starting parallel simulation for lengths: {ntiles_list}")
# Parallelize the outer loop over lengths
# This allows MD simulations to run in parallel which is much more efficient
avg_resistances = Parallel(n_jobs=-1)(
    delayed(run_simulation_for_length)(ntile) for ntile in tqdm(ntiles_list, desc="Processing Lengths")
)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(ntiles_list, avg_resistances, '-o', color='blue', label='MD Average')
plt.xlabel('Length (Unit Cells)')
plt.ylabel(r'Resistance ($\Omega$)')
plt.title(f'Resistance vs Length at T={md_temp_K}K')
plt.grid(True)
plt.legend()
plt.show()